In [1]:
import pandas as pd

In [2]:
seed_df = pd.read_csv('/blob/shufxi/data/scigpt/solubility_valid.csv')
seed_df.head()

,label,seq
0,0,SRLYRHNLMEDVFNMENESFMQETRLMENEYSVNLPTRFYYKKRWN...
1,1,ATTYNAVVSKSSSDGKTFKTIADAIASAPAGSTPFVILIKNGVYNE...
2,1,MEDQDLLEIGILNSGHRQRILQAIQLLPKMRPIGHDGYHPTSVAEW...
3,1,MGFAKPPWGGKLKGVVRILGGKARGVALKVPASARPSPVRLRKALF...
4,0,LAAGPALSARGYLALNGQTPAGCSLMEWQNDNNGRQRWCVRLVQGG...


In [3]:
pos_seqs = seed_df[seed_df['label'] == 1]['seq'].tolist()
neg_seqs = seed_df[seed_df['label'] == 0]['seq'].tolist()

print(len(pos_seqs), len(neg_seqs))

2897 4045


In [4]:
from sfm.data.sci_data.SFMDecTokenizer import SFMDecTokenizer
import torch
import os

/anaconda/envs/sfm2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-01-26 06:35:42,186] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [5]:
tokenizer_home = '/hai1/ds_dataset/llama2/llama-2-7b'
tokenizer = SFMDecTokenizer.from_pretrained(
    tokenizer_home,
    prot_spm_path='/blob/shufxi/data/scigpt/ur50bpe/bpe',
    dna_spm_path='/blob/shufxi/data/scigpt/dnabpe/bpe'
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'SFMDecTokenizer'.


You are using the default legacy behaviour of the <class 'sfm.data.sci_data.SFMDecTokenizer.SFMDecTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


[2024-01-26 06:35:43.911][INFO]: Loading protein sentencepiece model from /blob/shufxi/data/scigpt/ur50bpe/bpe.model and /blob/shufxi/data/scigpt/ur50bpe/bpe.vocab
[2024-01-26 06:35:43.969][INFO]: Loading DNA sentencepiece model from /blob/shufxi/data/scigpt/dnabpe/bpe.model and /blob/shufxi/data/scigpt/dnabpe/bpe.vocab
[2024-01-26 06:35:44.268][INFO]: Tokenizer has 38991 tokens


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
ckpt_home = '/hai1/shufxi/scigpt/7bv2/stageB/global_step26999/'

def show_ckpt(name, ckpt):
    for k, v in ckpt.items():
        if 'dummy' not in k:
            print(name, k, v.shape)

model = AutoModelForCausalLM.from_pretrained(tokenizer_home)

model_dict = model.state_dict()
ckpt_dict = {}
layer0 = torch.load(os.path.join(ckpt_home, "layer_00-model_states.pt"), map_location=torch.device("cpu"))
ckpt_dict['model.embed_tokens.weight'] = layer0['embed_tokens.weight']
show_ckpt('layer0', layer0)

for l in range(0, 32):
    l_index = str(l + 1).zfill(2)
    layer = torch.load(os.path.join(ckpt_home, f"layer_{l_index}-model_states.pt"), map_location=torch.device("cpu"))
    show_ckpt(l_index, layer)
    for k in layer:
        if "dummy" in k or 'rotary_emb' in k:
            continue
        ckpt_dict[f"model.layers.{l}.{k}"] = layer[k]
layer = torch.load(os.path.join(ckpt_home, "layer_33-model_states.pt"), map_location=torch.device("cpu"))
show_ckpt(33, layer)
ckpt_dict["model.norm.weight"] = layer["norm.weight"]

layer = torch.load(os.path.join(ckpt_home, "layer_34-model_states.pt"), map_location=torch.device("cpu"))
show_ckpt(33, layer)
ckpt_dict["lm_head.weight"] = layer["lm_head.weight"]
model_dict.update(ckpt_dict)

model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(model_dict)

Loading checkpoint shards: 100%|██████████| 2/2 [00:21<00:00, 10.62s/it]


layer0 embed_tokens.weight torch.Size([38991, 4096])
01 self_attn.q_proj.weight torch.Size([4096, 4096])
01 self_attn.k_proj.weight torch.Size([4096, 4096])
01 self_attn.v_proj.weight torch.Size([4096, 4096])
01 self_attn.o_proj.weight torch.Size([4096, 4096])
01 self_attn.rotary_emb.inv_freq torch.Size([64])
01 mlp.gate_proj.weight torch.Size([11008, 4096])
01 mlp.up_proj.weight torch.Size([11008, 4096])
01 mlp.down_proj.weight torch.Size([4096, 11008])
01 input_layernorm.weight torch.Size([4096])
01 post_attention_layernorm.weight torch.Size([4096])
02 self_attn.q_proj.weight torch.Size([4096, 4096])
02 self_attn.k_proj.weight torch.Size([4096, 4096])
02 self_attn.v_proj.weight torch.Size([4096, 4096])
02 self_attn.o_proj.weight torch.Size([4096, 4096])
02 self_attn.rotary_emb.inv_freq torch.Size([64])
02 mlp.gate_proj.weight torch.Size([11008, 4096])
02 mlp.up_proj.weight torch.Size([11008, 4096])
02 mlp.down_proj.weight torch.Size([4096, 11008])
02 input_layernorm.weight torch.Size

<All keys matched successfully>

In [7]:
model = model.cuda()

In [8]:
pos_gen, neg_gen = [], []

In [11]:
import random

In [12]:
from tqdm import tqdm

In [13]:
k = 3
target_n = 100
bar = tqdm(total=target_n)
pos_gen = []
while len(pos_gen) < target_n:
    prompt = "Protein solubility is the ability of a protein to dissolve in a solvent like water." + \
    "For example, a high solubility proteins can be "
    
    few_shot_examples = random.sample(pos_seqs, k)
    for example in few_shot_examples:
        prompt += "<protein>" + example + "</protein>" + ", "
    prompt += "and <protein>"
    
    input_ids = tokenizer.encode(prompt, return_tensors='pt').cuda()
    outputs = model.generate(input_ids, do_sample=True, top_p=0.9, max_new_tokens=300, num_return_sequences=1)
    output = outputs[0]
    
    # remove the prompt
    output = output[len(input_ids[0]):]
    output = tokenizer.decode(output)
    
    # find the content in the first <protein> </protein>
    end = output.find('</protein>')
    if end == -1:
        continue
    seq = output[:end]
    seq = seq.replace('<a>', '').replace(' ', '')
    if not seq:
        continue
    # check if already exists
    if seq in pos_seqs or seq in neg_seqs:
        continue
    pos_gen.append(seq)
    bar.update(1)
bar.close()


  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [15:28<00:00,  9.28s/it]


In [15]:
with open('positive.prot', 'w') as f:
    for seq in pos_gen:
        f.write(seq + '\n')

In [ ]:
k = 3
target_n = 100
bar = tqdm(total=target_n)
neg_gen = []
while len(neg_gen) < target_n:
    prompt = "Protein solubility is the ability of a protein to dissolve in a solvent like water." + \
    "For example, a low solubility proteins can be "
    
    few_shot_examples = random.sample(neg_seqs, k)
    for example in few_shot_examples:
        prompt += "<protein>" + example + "</protein>" + ", "
    prompt += "and <protein>"
    
    input_ids = tokenizer.encode(prompt, return_tensors='pt').cuda()
    outputs = model.generate(input_ids, do_sample=True, top_p=0.9, max_new_tokens=300, num_return_sequences=1)
    output = outputs[0]
    
    # remove the prompt
    output = output[len(input_ids[0]):]
    output = tokenizer.decode(output)
    
    # find the content in the first <protein> </protein>
    end = output.find('</protein>')
    if end == -1:
        continue
    seq = output[:end]
    seq = seq.replace('<a>', '').replace(' ', '')
    if not seq:
        continue
    # check if already exists
    if seq in pos_seqs or seq in neg_seqs:
        continue
    neg_gen.append(seq)
    bar.update(1)
bar.close()

with open('negative.prot', 'w') as f:
    for seq in neg_gen:
        f.write(seq + '\n')

100%|██████████| 100/100 [23:02<00:00, 13.83s/it]
